In [9]:
import pandas as pd

# Load the data from the Excel file
file_path = 'data/sub_comercial_bracatinga.xlsx'
data = pd.read_excel(file_path)
data = data
# Display the first few rows of the dataset to understand its structure
data.head()

,Trat,REP,IVE,EM%,GM%,PM%,SM%
1,T1R2,1,1.20,88.8,88.8,0.0,11.1
2,T1R3,1,1.56,100.0,100.0,0.0,0.0
3,T1R4,1,1.03,88.8,88.8,0.0,11.1
4,T1R5,1,0.80,77.7,77.7,0.0,33.3
5,T1R6,1,0.86,77.7,77.7,0.0,33.3


In [13]:
data_no_trat = data.drop("Trat", axis=1)

grouped_data = data_no_trat.groupby('REP').agg(['mean', 'std', 'max', 'min'])

grouped_data

IVE                              EM%                          \
         mean       std   max   min       mean        std    max   min   
REP                                                                      
1    1.090000  0.305614  1.56  0.80  86.600000   9.322821  100.0  77.7   
2    0.496667  0.114484  0.66  0.31  51.800000  11.464031   66.6  33.3   
3    0.735000  0.101341  0.91  0.63  79.566667  10.950738  100.0  66.6   
4    0.495000  0.174213  0.84  0.40  62.900000  13.442768   88.8  55.5   
5    0.620000  0.203175  0.97  0.42  70.316667  16.746990  100.0  55.5   
6    0.620000  0.203175  0.97  0.42  79.566667  13.007793  100.0  66.6   

           GM%                           PM%                          SM%  \
          mean        std    max   min  mean        std   max  min   mean   
REP                                                                         
1    86.600000   9.322821  100.0  77.7  0.00   0.000000   0.0  0.0  17.76   
2    46.233333  10.937215   55.5  33.2  5.55   9.286926  22.2  0.0  48.10   
3    72.166667  16.866970  100.0  55.5  7.40  11.464031  22.2  0.0  20.35   
4    62.900000  13.442768   88.8  55.5  0.00   0.000000   0.0  0.0  37.00   
5    68.466667  16.377261  100.0  55.5  1.85   4.531556  11.1  0.0  29.60   
6    79.566667  13.007793  100.0  66.6  0.00   0.000000   0.0  0.0  20.35   

                            
           std   max   min  
REP                         
1    14.892213  33.3   0.0  
2    11.464031  66.6  33.3  
3    10.913432  33.3   0.0  
4    13.442768  44.4  11.1  
5    16.711553  44.4   0.0  
6    12.976402  33.3   0.0

In [16]:
cv = (data_no_trat.drop('REP', axis=1).std() / data_no_trat.drop('REP', axis=1).mean()) * 100

cv

IVE     39.620942
EM%     23.368758
GM%     26.217881
PM%    261.763673
SM%     56.942536
dtype: float64

In [18]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# ANOVA
anova_model = ols('IVE ~ C(REP)', data=data_no_trat).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

anova_table

,sum_sq,df,F,PR(>F)
C(REP),1.300224,5.0,7.147924,0.000183
Residual,1.055033,29.0,NaN,NaN


In [19]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Primeiro, realizar ANOVA para verificar se há diferenças significativas entre os grupos
anova_model = ols('IVE ~ C(REP)', data=data_no_trat).fit()
anova_results = sm.stats.anova_lm(anova_model, typ=2)

# Se o ANOVA for significativo, proceder com o Teste de Tukey
if anova_results['PR(>F)'][0] < 0.05:  # Checando se o resultado é significativo
    tukey_results = pairwise_tukeyhsd(endog=data_no_trat['IVE'], groups=data_no_trat['REP'], alpha=0.05)
    print(tukey_results.summary())
else:
    print("ANOVA não significativo, não proceder com Tukey")

C:\Users\limei\AppData\Local\Temp\ipykernel_27360\2188840677.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if anova_results['PR(>F)'][0] < 0.05:  # Checando se o resultado é significativo


Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     1      2  -0.5933 0.0002 -0.9454 -0.2412   True
     1      3   -0.355 0.0472 -0.7071 -0.0029   True
     1      4   -0.595 0.0002 -0.9471 -0.2429   True
     1      5    -0.47 0.0041 -0.8221 -0.1179   True
     1      6    -0.47 0.0041 -0.8221 -0.1179   True
     2      3   0.2383 0.2845 -0.0974   0.574  False
     2      4  -0.0017    1.0 -0.3374   0.334  False
     2      5   0.1233 0.8691 -0.2124   0.459  False
     2      6   0.1233 0.8691 -0.2124   0.459  False
     3      4    -0.24 0.2776 -0.5757  0.0957  False
     3      5   -0.115 0.8988 -0.4507  0.2207  False
     3      6   -0.115 0.8988 -0.4507  0.2207  False
     4      5    0.125 0.8627 -0.2107  0.4607  False
     4      6    0.125 0.8627 -0.2107  0.4607  False
     5      6      0.0    1.0 -0.3357  0.3357  False
----------------------------------------------